# Reflection agents

In [ ]:
from langgraph.graph import MessageGraph, END
from langchain_core.messages import HumanMessage, AIMessage

# Node that generates an initial response
def generate_answer(state):
    # (In practice, call an LLM here)
    answer = "This is my first attempt."
    return {"messages": state["messages"] + [AIMessage(content=answer)]}

# Node that critiques and refines the previous answer
def critique_answer(state):
    # (In practice, call LLM to critique)
    critique = "The answer is incomplete; add more detail."
    return {"messages": state["messages"] + [AIMessage(content=critique)]}

builder = MessageGraph()
builder.add_node("generate", generate_answer)
builder.add_node("reflect", critique_answer)
builder.set_entry_point("generate")

# Loop control: alternate until max iterations
MAX_STEPS = 3
def should_continue(state):
    return "reflect" if len(state["messages"]) < 2*MAX_STEPS else END

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

# Run the reflection agent
initial_message = HumanMessage(content="Explain photosynthesis.")
result = graph.invoke({"messages": [initial_message]})
print(result["messages"][-1])  # Final answer or critique

# Reflexion agents

In [ ]:
from langgraph.graph import MessageGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
def draft_answer(state):
    # (LLM draft; could also generate search query)
    response = "The capital of France is Paris."
    return {"messages": state["messages"] + [AIMessage(content=response)]}
def execute_tools(state):
    # (Simulate external info; e.g., search results)
    info = "París (France) - capital: Paris (en.wikipedia.org)"
    return {"messages": state["messages"] + [SystemMessage(content=info)]}
def revise_answer(state):
    # (LLM re-evaluates answer using info)
    revision = "Yes, France’s capital is Paris. I've verified this."
    return {"messages": state["messages"] + [AIMessage(content=revision)]}
builder = MessageGraph()
builder.add_node("draft", draft_answer)
builder.add_node("execute_tools", execute_tools)
builder.add_node("revise", revise_answer)
builder.add_edge("draft", "execute_tools")
builder.add_edge("execute_tools", "revise")
# Loop control: stop after N iterations
MAX_LOOPS = 2
def continue_reflexion(state):
    # Count assistant messages to determine iteration
    iteration = sum(1 for m in state["messages"] if isinstance(m, AIMessage))
    return "execute_tools" if iteration <= MAX_LOOPS else END
builder.add_conditional_edges("revise", continue_reflexion)
builder.set_entry_point("draft")
graph = builder.compile()
initial_message = HumanMessage(content="What is the capital of France?")
result = graph.invoke({"messages": [initial_message]}) # Final revised answer

# ReAct agents

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage

# Simple state with messages and a step counter
def call_model(state):
    # (LLM reasons; may request an action or give an answer)
    last = state["messages"][-1]
    if "weather" in last:
        # chain-of-thought leading to an action
        thought = AIMessage(content="Let me find the weather for you.")
        return {"messages": state["messages"] + [thought]}
    else:
        # final answer
        answer = AIMessage(content="It's sunny in NYC today.")
        return {"messages": state["messages"] + [answer]}

def call_tool(state):
    # (Simulate a weather API/tool result)
    tool_result = AIMessage(content="Weather(temperature=75F, condition=sunny)")
    return {"messages": state["messages"] + [tool_result]}

# Decide whether to act or finish based on last message
def next_step(state):
    last = state["messages"][-1]
    if "find the weather" in last:
        return "tools"
    return "end"

graph = StateGraph(dict)  # using a plain dict state
graph.add_node("think", call_model)
graph.add_node("act", call_tool)
graph.set_entry_point("think")
# If the model’s message triggers an action, go to 'act'; else end.
graph.add_conditional_edges("think", next_step, {"tools": "act", "end": END})
graph.add_edge("act", "think")
compiled = graph.compile()

result = compiled.invoke({"messages": [HumanMessage(content="What is the weather in NYC?")]})
print(result["messages"][-1])  # Final assistant answer


## Comparison of Agent Styles

| Aspect | Reflection Agent | Reflexion Agent | ReAct Agent |
|--------|------------------|-----------------|-------------|
| **Core idea** | Model critiques its own answer | Model critiques with external feedback and citations | Model reasons and acts (calls tools) in loop |
| **Structure** | Generator → Reflector → (loop) | Draft → (Search/Tool) → Revisor → (loop) | LLM → (conditional Tool call) → LLM → … |
| **Graph components** | 2 nodes (generate, reflect) | 3+ nodes (draft, execute tools, revise) | 2 nodes (think, act) with conditional branching |
| **Feedback source** | Internal (LLM self-review) | External (tool or search results + LLM review) | External (tool calls informed by model reasoning) |
| **Benefits** | Simple setup; improves coherence & detail | High accuracy; enforces evidence and completeness | Flexible tool use; handles complex tasks |
| **Drawbacks** | May plateau (no new info); extra compute | More complex and slow (searches/tools each loop) | Requires designing tools; complexity in prompts |
| **Use cases** | Refining essays, content drafts | Fact-checking, coding, QA with citations | Question answering with APIs, step-by-step tasks |
